In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Load in csv from part 1
city_data= pd.read_csv("../WeatherPy/city_data.csv")
city_data.dropna(inplace = True)
city_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mahébourg,-20.4081,57.7000,67.39,62,60,24.25,MU,1595527200
1,1,Busselton,-33.6500,115.3333,52.09,78,5,3.27,AU,1595527200
2,2,Stornoway,58.2093,-6.3865,58.82,78,88,8.93,GB,1595527200
3,3,Rikitea,-23.1203,-134.9692,66.99,66,3,18.21,PF,1595527200
4,4,Katsuura,35.1333,140.3000,72.81,92,100,1.74,JP,1595527200
...,...,...,...,...,...,...,...,...,...,...
564,564,Lata,40.1629,-8.3327,79.90,52,0,7.72,PT,1595527200
565,565,Jidong,45.2167,131.0833,64.11,89,0,1.32,CN,1595527200
566,566,Kahului,20.8947,-156.4700,78.17,71,12,10.09,US,1595527200
567,567,Poronaysk,49.2167,143.1167,52.36,88,100,2.19,RU,1595527200


In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations= city_data[["Lat", "Lng"]]
humidity= city_data["Humidity"].astype(float)

In [5]:
# Add Heatmap layer to map.
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=3)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Create new DataFrame to narrow down cities by ideal weather conditions 
# Max temperature lower than 80 but higher than 70. 
ideal_conditions= city_data.loc[city_data["Max Temp"]<80,:]
ideal_conditions = ideal_conditions.loc[ideal_conditions["Max Temp"]>70,:]

# Wind speed less than 10 mph.
ideal_conditions = ideal_conditions.loc[ideal_conditions["Wind Speed"]<10,:]

#Zero cloudiness.
ideal_conditions = ideal_conditions.loc[ideal_conditions["Cloudiness"]==0,:]
ideal_conditions.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,Ponta do Sol,32.6667,-17.1000,76.39,66,0,1.03,PT,1595527200
36,36,Mossamedes,-15.1961,12.1522,73.94,57,0,4.85,AO,1595527200
135,135,Masallātah,32.6167,14.0000,78.46,62,0,9.37,LY,1595527200
173,173,Pánormos,35.4182,24.6909,79.16,58,0,5.39,GR,1595527200
392,392,Kiryat Gat,31.6100,34.7642,78.33,74,0,4.03,IL,1595527200


In [7]:

#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.

In [8]:
#Hotel Map
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
hotel_df['City'] = ideal_conditions['City']
hotel_df['Country'] = ideal_conditions['Country']
hotel_df['Lat'] = ideal_conditions['Lat']
hotel_df['Lng'] = ideal_conditions['Lng']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
11,Ponta do Sol,PT,32.6667,-17.1000
36,Mossamedes,AO,-15.1961,12.1522
135,Masallātah,LY,32.6167,14.0000
173,Pánormos,GR,35.4182,24.6909
392,Kiryat Gat,IL,31.6100,34.7642


In [9]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 11: Ponta do Sol.
Closest hotel to Ponta do Sol at 32.6667 , -17.1 is Casa Alto Santo António.
------------
Retrieving Results for Index 36: Mossamedes.
Closest hotel to Mossamedes at -15.1961 , 12.1522 is Banco Millennium Atlântico.
------------
Retrieving Results for Index 135: Masallātah.
Closest hotel to Masallātah at 32.6167 , 14.0 is Mustafa Taher Sowaidan.
------------
Retrieving Results for Index 173: Pánormos.
Closest hotel to Pánormos at 35.4182 , 24.6909 is hotel Iliana.
------------
Retrieving Results for Index 392: Kiryat Gat.
Closest hotel to Kiryat Gat at 31.61 , 34.7642 is Shupersol Kiryat Gat.
------------
Retrieving Results for Index 413: Yebaishou.
Closest hotel to Yebaishou at 41.3975 , 119.6408 is Jianping Labor Employment Bureau.
------------
Retrieving Results for Index 449: Pedra Azul.
Closest hotel to Pedra Azul at -16.0053 , -41.2972 is BANCO DO BRASIL - PEDRA AZUL.
------------
Retrieving Results for Index 564: Lata.
Closest hotel

In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
11,Ponta do Sol,PT,32.6667,-17.1000,Casa Alto Santo António
36,Mossamedes,AO,-15.1961,12.1522,Banco Millennium Atlântico
135,Masallātah,LY,32.6167,14.0000,Mustafa Taher Sowaidan
173,Pánormos,GR,35.4182,24.6909,hotel Iliana
392,Kiryat Gat,IL,31.6100,34.7642,Shupersol Kiryat Gat
413,Yebaishou,CN,41.3975,119.6408,Jianping Labor Employment Bureau
449,Pedra Azul,BR,-16.0053,-41.2972,BANCO DO BRASIL - PEDRA AZUL
564,Lata,PT,40.1629,-8.3327,Atocentro Fabrico De Acessorios Para A Agricul...


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [12]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=3)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))